In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [6]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load GPT-2 model and tokenizer
model_name = "gpt2"  # You can also use other variants like "gpt2-medium", "gpt2-large", etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)




config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

C:\Users\ASUS\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you?

I'm a little bit of a nerd. I'm a big nerd. I'm a


In [9]:
# Example text
input_text = "Hello, how are you?"
print(input_text)

# Tokenize input text
inputs = tokenizer(input_text, return_tensors="pt")
print(inputs) 

# Generate output
#outputs = model.generate(**inputs)
#outputs = model.generate(**inputs, do_sample=True, top_p=0.95, temperature=0.7)
outputs = model.generate(**inputs, max_length=200, no_repeat_ngram_size=2, do_sample=True, top_p=0.95, temperature=0.7)

print(outputs) 

# Decode the generated tokens to text
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(output_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you?
{'input_ids': tensor([[15496,    11,   703,   389,   345,    30]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}
tensor([[15496,    11,   703,   389,   345,    30,   314,   892,   356,   460,
           466,   428,  1978,   553,   673,   531,    13,   198,   198,  3347,
           788,  1234,   866,   607,  6131,   290,  1718,   257,  5852,   379,
           262,  3084,    11,  5586,   866,   351,   257,  8212,   319,   607,
          1986,    13,   366,  1135,  1183,   766,   703,   345,   821,  1804,
           553,   531,   530,   286,   262,  1854,    13, 50256]])
Hello, how are you? I think we can do this together," she said.

She then put down her bag and took a seat at the table, sitting down with a smile on her face. "We'll see how you're doing," said one of the others.


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "gpt2"

# Load GPT-2 model (quantized for low memory)
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Save model in chunks for splitting (if necessary)
torch.save(model.state_dict(), "gpt2_model_split.pth")
# Get the state dictionary of the model
state_dict = model.state_dict()

# Print out the keys to see the model layers (for splitting)
for key in state_dict.keys():
    print(key)

transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

In [14]:
# Assuming you have 12 transformer layers, split them into 3 parts (4 layers per part)
part1 = {}
part2 = {}
part3 = {}

# Split transformer layers into 3 parts
layers = [key for key in state_dict.keys() if "transformer.h." in key]

# Divide layers into three parts
part1_layers = layers[:4]
part2_layers = layers[4:8]
part3_layers = layers[8:]

# Add layers to each part
for layer in part1_layers:
    part1[layer] = state_dict[layer]

for layer in part2_layers:
    part2[layer] = state_dict[layer]

for layer in part3_layers:
    part3[layer] = state_dict[layer]

# Save each part
torch.save(part1, "gpt2_part1.pth")
torch.save(part2, "gpt2_part2.pth")
torch.save(part3, "gpt2_part3.pth")


In [15]:
# Get all transformer blocks
transformer_blocks = model.transformer.h

# Split model into two halves
midpoint = len(transformer_blocks) // 2
part1_blocks = transformer_blocks[:midpoint]
part2_blocks = transformer_blocks[midpoint:]

# Create two separate model parts
model_part1 = torch.nn.Sequential(*part1_blocks)
model_part2 = torch.nn.Sequential(*part2_blocks)

# Save the two parts separately
torch.save(model_part1.state_dict(), "gpt2_part1_taher.pth")
torch.save(model_part2.state_dict(), "gpt2_part2_rabiul.pth")

print("Model successfully split into two parts.")


Model successfully split into two parts.


In [23]:
from transformers import GPT2Model
import torch

# Load the GPT-2 model
model = GPT2Model.from_pretrained("gpt2")
print("model:\n", model)

# Split the transformer blocks
transformer_blocks = model.h  # Access the transformer blocks directly
print("transformer block: \n", transformer_blocks)

# Find the midpoint for splitting
midpoint = len(transformer_blocks) // 2
print("midpoint: ", midpoint)

# Split the blocks into two parts
part1_blocks = transformer_blocks[:midpoint]
part2_blocks = transformer_blocks[midpoint:]
print("block 1: \n", part1_blocks)
print("block 2: \n", part2_blocks)

# Save each part
model_part1 = GPT2Model.from_pretrained("gpt2")
print("model part 1: \n", model_part1)
model_part1.h = torch.nn.ModuleList(part1_blocks)  # Replace with the first part
torch.save(model_part1.state_dict(), "gpt2_part1_taher.pth")
print("model part 1: \n", model_part1)


model_part2 = GPT2Model.from_pretrained("gpt2")
print("model part 2: \n", model_part2)
model_part2.h = torch.nn.ModuleList(part2_blocks)  # Replace with the second part
torch.save(model_part2.state_dict(), "gpt2_part2_rabiul.pth")
print("model part 2: \n", model_part2)

model:
 GPT2Model(
  (wte): Embedding(50257, 768)
  (wpe): Embedding(1024, 768)
  (drop): Dropout(p=0.1, inplace=False)
  (h): ModuleList(
    (0-11): 12 x GPT2Block(
      (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (attn): GPT2Attention(
        (c_attn): Conv1D(nf=2304, nx=768)
        (c_proj): Conv1D(nf=768, nx=768)
        (attn_dropout): Dropout(p=0.1, inplace=False)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
      (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (mlp): GPT2MLP(
        (c_fc): Conv1D(nf=3072, nx=768)
        (c_proj): Conv1D(nf=768, nx=3072)
        (act): NewGELUActivation()
        (dropout): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
)
transformer block: 
 ModuleList(
  (0-11): 12 x GPT2Block(
    (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (attn): GPT2Attention(
      (c_attn): Conv1D(nf=2304, nx=768)
    

In [21]:
from transformers import GPT2Model
import torch

# Load the architecture for the first part of the model
class Part1Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Load the full GPT-2 model architecture
        self.model = GPT2Model.from_pretrained("gpt2")
        # Retain only the first half of the transformer blocks
        self.model.h = torch.nn.ModuleList(
            list(self.model.h)[:len(self.model.h) // 2]
        )

    def forward(self, x):
        return self.model(x)[0]

# Instantiate the first part of the model
model_part1 = Part1Model()

# Load the weights for the first part
model_part1.load_state_dict(torch.load("gpt2_part1_taher.pth"))

print("Part 1 model loaded successfully!")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4324\2623574572.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_part1.load_state_dict(torch.load("gpt2_part1_taher.pth"

RuntimeError: Error(s) in loading state_dict for Part1Model:
	Missing key(s) in state_dict: "model.wte.weight", "model.wpe.weight", "model.h.0.ln_1.weight", "model.h.0.ln_1.bias", "model.h.0.attn.c_attn.weight", "model.h.0.attn.c_attn.bias", "model.h.0.attn.c_proj.weight", "model.h.0.attn.c_proj.bias", "model.h.0.ln_2.weight", "model.h.0.ln_2.bias", "model.h.0.mlp.c_fc.weight", "model.h.0.mlp.c_fc.bias", "model.h.0.mlp.c_proj.weight", "model.h.0.mlp.c_proj.bias", "model.h.1.ln_1.weight", "model.h.1.ln_1.bias", "model.h.1.attn.c_attn.weight", "model.h.1.attn.c_attn.bias", "model.h.1.attn.c_proj.weight", "model.h.1.attn.c_proj.bias", "model.h.1.ln_2.weight", "model.h.1.ln_2.bias", "model.h.1.mlp.c_fc.weight", "model.h.1.mlp.c_fc.bias", "model.h.1.mlp.c_proj.weight", "model.h.1.mlp.c_proj.bias", "model.h.2.ln_1.weight", "model.h.2.ln_1.bias", "model.h.2.attn.c_attn.weight", "model.h.2.attn.c_attn.bias", "model.h.2.attn.c_proj.weight", "model.h.2.attn.c_proj.bias", "model.h.2.ln_2.weight",

In [22]:
from transformers import GPT2Model
import torch

# Load the architecture for the second part of the model
class Part2Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # Load the full GPT-2 model architecture
        self.model = GPT2Model.from_pretrained("gpt2")
        # Retain only the second half of the transformer blocks
        self.model.h = torch.nn.ModuleList(
            list(self.model.h)[len(self.model.h) // 2:]
        )

    def forward(self, x):
        return self.model(x)[0]

# Instantiate the second part of the model
model_part2 = Part2Model()

# Load the weights for the second part
model_part2.load_state_dict(torch.load("gpt2_part2_rabiul.pth"))

print("Part 2 model loaded successfully!")


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4324\1124008301.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_part2.load_state_dict(torch.load("gpt2_part2_rabiul.pth

RuntimeError: Error(s) in loading state_dict for Part2Model:
	Missing key(s) in state_dict: "model.wte.weight", "model.wpe.weight", "model.h.0.ln_1.weight", "model.h.0.ln_1.bias", "model.h.0.attn.c_attn.weight", "model.h.0.attn.c_attn.bias", "model.h.0.attn.c_proj.weight", "model.h.0.attn.c_proj.bias", "model.h.0.ln_2.weight", "model.h.0.ln_2.bias", "model.h.0.mlp.c_fc.weight", "model.h.0.mlp.c_fc.bias", "model.h.0.mlp.c_proj.weight", "model.h.0.mlp.c_proj.bias", "model.h.1.ln_1.weight", "model.h.1.ln_1.bias", "model.h.1.attn.c_attn.weight", "model.h.1.attn.c_attn.bias", "model.h.1.attn.c_proj.weight", "model.h.1.attn.c_proj.bias", "model.h.1.ln_2.weight", "model.h.1.ln_2.bias", "model.h.1.mlp.c_fc.weight", "model.h.1.mlp.c_fc.bias", "model.h.1.mlp.c_proj.weight", "model.h.1.mlp.c_proj.bias", "model.h.2.ln_1.weight", "model.h.2.ln_1.bias", "model.h.2.attn.c_attn.weight", "model.h.2.attn.c_attn.bias", "model.h.2.attn.c_proj.weight", "model.h.2.attn.c_proj.bias", "model.h.2.ln_2.weight",

In [24]:
from transformers import GPT2Model
import torch

# Load the full GPT-2 model
full_model = GPT2Model.from_pretrained("gpt2")

# Split the transformer blocks
transformer_blocks = full_model.h
midpoint = len(transformer_blocks) // 2

# First part: Embeddings + First half of transformer blocks
class GPT2Part1(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.wte = original_model.wte  # Word embeddings
        self.wpe = original_model.wpe  # Positional embeddings
        self.drop = original_model.drop  # Dropout
        self.h = torch.nn.ModuleList(original_model.h[:midpoint])  # First half of transformer blocks

    def forward(self, input_ids, attention_mask=None):
        # Compute embeddings
        input_embeds = self.wte(input_ids) + self.wpe(torch.arange(input_ids.size(1), device=input_ids.device))
        hidden_states = self.drop(input_embeds)

        # Pass through the first half of the transformer blocks
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)

        return hidden_states


# Second part: Second half of transformer blocks + Final layer norm
class GPT2Part2(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.h = torch.nn.ModuleList(original_model.h[midpoint:])  # Second half of transformer blocks
        self.ln_f = original_model.ln_f  # Final layer norm

    def forward(self, hidden_states, attention_mask=None):
        # Pass through the second half of the transformer blocks
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)

        # Apply final layer norm
        output_states = self.ln_f(hidden_states)
        return output_states


# Create part 1 and part 2
model_part1 = GPT2Part1(full_model, midpoint)
model_part2 = GPT2Part2(full_model, midpoint)

# Save each part
torch.save(model_part1.state_dict(), "gpt2_part1_taher2.pth")
torch.save(model_part2.state_dict(), "gpt2_part2_rabiul2.pth")


In [25]:
import torch
from transformers import GPT2Tokenizer, GPT2Model

# Define the GPT2Part1 and GPT2Part2 classes (reuse the classes from the splitting code)
class GPT2Part1(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.wte = original_model.wte
        self.wpe = original_model.wpe
        self.drop = original_model.drop
        self.h = torch.nn.ModuleList(original_model.h[:midpoint])

    def forward(self, input_ids, attention_mask=None):
        input_embeds = self.wte(input_ids) + self.wpe(torch.arange(input_ids.size(1), device=input_ids.device))
        hidden_states = self.drop(input_embeds)
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)
        return hidden_states


class GPT2Part2(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.h = torch.nn.ModuleList(original_model.h[midpoint:])
        self.ln_f = original_model.ln_f

    def forward(self, hidden_states, attention_mask=None):
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)
        output_states = self.ln_f(hidden_states)
        return output_states


# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load the saved models
full_model = GPT2Model.from_pretrained("gpt2")
midpoint = len(full_model.h) // 2

model_part1 = GPT2Part1(full_model, midpoint)
model_part1.load_state_dict(torch.load("gpt2_part1_taher2.pth"))

model_part2 = GPT2Part2(full_model, midpoint)
model_part2.load_state_dict(torch.load("gpt2_part2_rabiul2.pth"))

# Put the models in evaluation mode
model_part1.eval()
model_part2.eval()

# Example input text
text = "The quick brown fox jumps over the lazy dog."

# Tokenize input
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]

# Step 1: Pass input through part 1
hidden_states = model_part1(input_ids)

# Step 2: Pass output of part 1 to part 2
output_states = model_part2(hidden_states)

# Print the shape of the final output
print("Final output shape:", output_states.shape)

# Optional: Decode the output (depends on your usage, may require fine-tuning for text generation)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4324\1669493709.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_part1.load_state_dict(torch.load("gpt2_part1_taher2.pth

TypeError: layer_norm(): argument 'input' (position 1) must be Tensor, not tuple

In [26]:
import torch
from transformers import GPT2Model, GPT2Tokenizer

# Define the first part of GPT-2
class GPT2Part1(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.wte = original_model.wte  # Word embeddings
        self.wpe = original_model.wpe  # Positional embeddings
        self.drop = original_model.drop  # Dropout
        self.h = torch.nn.ModuleList(original_model.h[:midpoint])  # First half of transformer blocks
        self.ln_f = original_model.ln_f  # Final normalization layer (for splitting)

    def forward(self, input_ids, attention_mask=None):
        # Compute embeddings
        input_embeds = self.wte(input_ids) + self.wpe(torch.arange(input_ids.size(1), device=input_ids.device))
        hidden_states = self.drop(input_embeds)

        # Pass through the first half of the transformer blocks
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)

        # Apply normalization before passing to the second part
        hidden_states = self.ln_f(hidden_states)
        return hidden_states


# Define the second part of GPT-2
class GPT2Part2(torch.nn.Module):
    def __init__(self, original_model, midpoint):
        super().__init__()
        self.h = torch.nn.ModuleList(original_model.h[midpoint:])  # Second half of transformer blocks
        self.ln_f = original_model.ln_f  # Final normalization layer (retained in case of final output)

    def forward(self, hidden_states, attention_mask=None):
        # Pass through the second half of the transformer blocks
        for block in self.h:
            hidden_states = block(hidden_states, attention_mask=attention_mask)

        # Final normalization
        output_states = self.ln_f(hidden_states)
        return output_states


# Load the original GPT-2 model
full_model = GPT2Model.from_pretrained("gpt2")

# Split the model
midpoint = len(full_model.h) // 2
model_part1 = GPT2Part1(full_model, midpoint)
model_part2 = GPT2Part2(full_model, midpoint)

# Save the models
torch.save(model_part1.state_dict(), "gpt2_part111.pth")
torch.save(model_part2.state_dict(), "gpt2_part222.pth")


In [27]:
# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Load split models
model_part1.load_state_dict(torch.load("gpt2_part111.pth"))
model_part2.load_state_dict(torch.load("gpt2_part222.pth"))

# Set models to evaluation mode
model_part1.eval()
model_part2.eval()

# Example input text
text = "The quick brown fox jumps over the lazy dog."
inputs = tokenizer(text, return_tensors="pt")
input_ids = inputs["input_ids"]
print(input_ids) 

# Inference: Part 1
hidden_states = model_part1(input_ids)

# Inference: Part 2
output_states = model_part2(hidden_states)

# Print the output shape
print("Final output shape:", output_states.shape)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_4324\3452869986.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_part1.load_state_dict(torch.load("gpt2_part111.pth"))
C:

TypeError: layer_norm(): argument 'input' (position 1) must be Tensor, not tuple